In [1]:
from pandas_datareader import data # 야후 파이낸스에서 자료 가져오기
from dateutil.relativedelta import relativedelta  # 상대적 날짜 구하기
import pandas as pd # 데이터프레임 만들기 등

In [2]:
start_day = '2000-01-01'
end_day = '2020-02-28'

In [3]:
# 거래일 리스트 만들기
date = []
date_index = data.get_data_yahoo('^IXIC', start_day, end_day).index
for i in range(len(date_index)):
    date.append(date_index[i])

# 나스닥 지수 불러오기
nasdaq_price = data.get_data_yahoo('^IXIC', start_day, end_day)['Adj Close'].to_list()

nasdaq_return = []
nasdaq_return.append(0)
for i in range(1, len(date)):
    nasdaq_return_tmp = (nasdaq_price[i] / nasdaq_price[i - 1] - 1) * 100
    nasdaq_return.append(nasdaq_return_tmp)

# 채권 가격, 등락률 불러오기
treasury_price = data.get_data_yahoo('VUSTX', start_day, end_day)['Adj Close'].to_list()

treasury_return = []
treasury_return.append(0)
for i in range(1, len(date)):
    treasury_return_tmp = (treasury_price[i] / treasury_price[i - 1] - 1) * 100
    treasury_return.append(treasury_return_tmp)

# VIX 지수 추가
vix_index = data.get_data_yahoo('^VIX', start_day, end_day)['Adj Close'].to_list()

print('지수 데이터 생성 완료') 

지수 데이터 생성 완료


In [4]:
# 종목 리스트 
tickers = ['AAPL', 'AMZN', 'BABA', 'BAC', 'BHP', 'BP', 'BRK-A', 'C', 'CHL', 'CICHF', 'CSCO', 'CVX', 'FB', 'GE', 'GOOGL',
           'HSBC', 'IBM', 'IDCBF', 'JNJ', 'MSFT', 'NSRGF', 'NVS', 'OGZPY', 'PBR', 'PFE', 'PG', 'PTR', 'RDS-B', 'RHHVF',
           'T', 'TM', 'WFC', 'WMT', 'XOM']

# 데이터프레임 생성
price_df = pd.DataFrame()  # 주가 데이터프레임 생성
price_df['date'] = date
price_df.set_index('date', inplace=True)
price_df['nasdaq_price'] = nasdaq_price
price_df['treasury_price'] = treasury_price

return_df = pd.DataFrame()  # 등락률 데이터프레임 생성
return_df['date'] = date
return_df.set_index('date', inplace=True)
return_df['nasdaq_return'] = nasdaq_return
return_df['treasury_return'] = treasury_return

marketcap_df = pd.DataFrame()  # 시가총액 데이터프레임 생성
marketcap_df['date'] = date
marketcap_df.set_index('date', inplace=True)

# 종목별 주가, 수익률 추가
for i in range(0, len(tickers)): # 티커를 돌면서
    ticker = tickers[i]
    price = ticker + '_price'
    daily_return = ticker + '_return'
    marketcap = ticker + '_marketcap'

    # 거래일 맞춰서 주가 입력
    price_data = data.get_data_yahoo(ticker, start_day, end_day) # 야후 파이낸스에서 종목별 가격 데이터 추출하기
    locals()[price] = []
    for j in range(0, len(date)): # 거래일별로 돌면서
        try:
            price_tmp = price_data.loc[date[j]]['Adj Close'] # 거래일자별 수정주가 호출
        except: # 만약 거래일에 거래 정보가 없어서 에러가 뜨면
            if j == 0: # 첫 거래일일 경우
                price_tmp = 0 # 가격은 0으로 처리 (아직 상장 전)
            else: # 나머지 거래일일 경우
                price_tmp = locals()[price][j - 1] # 가격은 전일 거래일로 처리
        locals()[price].append(price_tmp) # 리스트에 데이터 입력
    price_df[ticker + '_price'] = locals()[price] # 데이터프레임에 가격 리스트 입력

    # 수익률 입력
    locals()[daily_return] = []
    locals()[daily_return].append(0) # 첫 거래일 수익률은 0 으로 처리
    for j in range(1, len(date)): # 두번째 거래일 부터 나머지 거래일을 돌면서
        if locals()[price][j - 1] == 0: # 전일 가격 정보가 0이면 (아직 상장 전)
            return_tmp = 0 # 수익률 0으로 처리
        else: # 전일 가격 정보가 있으면 
            return_tmp = (locals()[price][j] / locals()[price][j - 1] - 1) * 100 # 당일 가격을 전일 가격으로 나눠서 수익률 계산
        locals()[daily_return].append(return_tmp) # 리스트에 데이터 입력
    return_df[ticker + '_return'] = locals()[daily_return] # 데이터프레임에 수익률 리스트 입력

    # 시가총액 입력
    locals()[marketcap] = []
    marketcap_data = pd.read_csv('marketcap.csv') # 시가총액 파일 열어서 그대로 임시 데이터프레임 생성 
    marketcap_data.set_index('Date', inplace=True)

    for j in range(0, len(date)):
        marketcap_tmp = marketcap_data.loc[date[j].strftime('%Y-%m-%d'), tickers[i]]
        locals()[marketcap].append(marketcap_tmp)
    marketcap_df[ticker + '_marketcap'] = locals()[marketcap]

    print(str(i + 1) + '종목/' + str(len(tickers)) + '종목 데이터 생성 완료') # 진행상태 표시용 출력

1종목/34종목 데이터 생성 완료
2종목/34종목 데이터 생성 완료
3종목/34종목 데이터 생성 완료
4종목/34종목 데이터 생성 완료
5종목/34종목 데이터 생성 완료
6종목/34종목 데이터 생성 완료
7종목/34종목 데이터 생성 완료
8종목/34종목 데이터 생성 완료
9종목/34종목 데이터 생성 완료
10종목/34종목 데이터 생성 완료
11종목/34종목 데이터 생성 완료
12종목/34종목 데이터 생성 완료
13종목/34종목 데이터 생성 완료
14종목/34종목 데이터 생성 완료
15종목/34종목 데이터 생성 완료
16종목/34종목 데이터 생성 완료
17종목/34종목 데이터 생성 완료
18종목/34종목 데이터 생성 완료
19종목/34종목 데이터 생성 완료
20종목/34종목 데이터 생성 완료
21종목/34종목 데이터 생성 완료
22종목/34종목 데이터 생성 완료
23종목/34종목 데이터 생성 완료
24종목/34종목 데이터 생성 완료
25종목/34종목 데이터 생성 완료
26종목/34종목 데이터 생성 완료
27종목/34종목 데이터 생성 완료
28종목/34종목 데이터 생성 완료
29종목/34종목 데이터 생성 완료
30종목/34종목 데이터 생성 완료
31종목/34종목 데이터 생성 완료
32종목/34종목 데이터 생성 완료
33종목/34종목 데이터 생성 완료
34종목/34종목 데이터 생성 완료


In [5]:
# 종목별 시총 순위 구하기
marketcap_ranking = marketcap_df.rank(ascending=False, method='first', axis=1) # 시총 데이터프레임의 일자별 시총 랭킹 구하기

# 시총 1, 2위 종목 구하기
rank1 = []
rank1_return = []
rank2 = []
rank2_return = []

for i in range(0, len(date)): # 일자별로 돌면서
    for j in range(0, len(tickers)): # 종목별로 돌면서
        if marketcap_ranking.iloc[i, j] == 1: # 랭킹 1위인 종목 찾아서
            rank1.append(tickers[j]) # 랭크1 리스트에 추가
        elif marketcap_ranking.iloc[i, j] == 2: # 랭킹 2위인 종목 찾아서
            rank2.append(tickers[j]) # 랭크2 리스트에 추가

marketcap_ranking['rank1'] = rank1
marketcap_ranking['rank2'] = rank2
print('시총 순위 데이터 생성 완료')

# 1위, 2위 차이, 수익률 구하기
rank_distance = []
for i in range(0, len(date)): # 일자별로 돌면서
    rank_distance.append((marketcap_df.loc[date[i], marketcap_ranking.iloc[i, -2] + '_marketcap'] - marketcap_df.loc[
        date[i], marketcap_ranking.iloc[i, -1] + '_marketcap']) / marketcap_df.loc[
                             date[i], marketcap_ranking.iloc[i, -1] + '_marketcap'] * 100) # (랭크1 종목 시총 - 랭크2 종목 시총)/랭크2 종목 시총으로 시총 차이 계산
marketcap_ranking['rank_distance'] = rank_distance 

for i in range(0, len(date)): # 일자별로 돌면서
    rank1_return.append(locals()[rank1[i] + '_return'][i]) # 시총 1위 종목 수익률 찾아서 리스트 추가
    rank2_return.append(locals()[rank2[i] + '_return'][i]) # 시총 2위 종목 수익률 찾아서 리스트 추가
marketcap_ranking['rank1_return'] = rank1_return
marketcap_ranking['rank2_return'] = rank2_return
print('시총 1, 2위 데이터 생성 완료')

시총 순위 데이터 생성 완료
시총 1, 2위 데이터 생성 완료


In [6]:
# 리스트 및 초기 변수 생성
strategy = []
market_status = []
recovery = []
depression = []
return_day = []
depression_signal = 0  # 공황 카운트
recovery_signal = 0  # 회복 카운트

# 첫 거래일
market_status.append('평화') # 첫 거래일의 시장 상태는 '평화'로 설정
if nasdaq_return[0] <= -3:  # 나스닥이 -3%가 뜨면
    strategy.append('탈출')  # 탈출 전략 가동하고
    depression_signal = depression_signal + 1  # 공황 시그널 1추가
    return_date = date[0] + relativedelta(months=1)  # 탈출 날짜 기준 한 달 뒤로 복귀 기준 날짜 설정
else: # 나스닥이 -3%가 안뜨면
    strategy.append('1등주')  # 1등주 보유
    return_date = date[0] # 복귀 기준일은 당일로 초기화
recovery.append(recovery_signal) 
depression.append(depression_signal)
return_day.append(return_date)

# 이후 거래일
for i in range(1, len(date)): # 거래 2일차 부터 나머지 거래일을 돌면서
    # 복귀일 처리
    if date[i] >= return_date + relativedelta(days=1):  # 복귀일(한달 뒤 다음 날)일 때
        if market_status[i - 1] != '공황':  # 전날이 공황상태가 아니면
            depression_signal = 0  # 공황시그널 초기화
            recovery_signal = 0  # 회복시그널 초기화

        else:  # 전날이 공황상태면 (복귀 기간까지 -3%가 안떴으니 시장 상태 회복)
            depression_signal = 0  # 공황시그널 초기화
            recovery_signal = 1  # 회복시그널 초기화

    # 전략 처리
    if nasdaq_return[i] <= -3:  # 나스닥이 -3%가 뜨면
        strategy.append('탈출')  # 탈출 전략 가동하고
        if market_status[i - 1] == '공황':  # 전 날이 공황상태일 때
            if recovery_signal == 1:  # 회복 시그널이 떴으면 (평화상태 의미)
                depression_signal = depression_signal + 1  # 공황 시그널 1추가
                recovery_signal = 0  # 회복시그널 초기화
                return_date = date[i] + relativedelta(months=1)  # 탈출 날짜 기준 한 달 뒤 복귀 날짜 설정
                market_status.append('평화')
            else:  # 회복 시그널이 안떴으면, (계속 공황상태 의미)
                depression_signal = depression_signal + 1  # 공황 시그널 1추가
                return_date = date[i] + relativedelta(months=2)  # 탈출 날짜 기준 두 달 뒤 복귀 날짜 설정
                market_status.append('공황')

        else:  # 전 날이 평화상태일 때
            drop_count = 0  # 기간내 하락 횟수값 초기화
            last_month = date[0] # 전월 거래일 날짜 초기화

            for j in range(0, date.index(date[i - 1])):  # 첫 거래일부터 전 날 까지 돌면서
                if date[j] <= date[i - 1] + relativedelta(months=-1):  # 한 달 전 날짜 보다 작거나 같으면 (정확히 한 달 전 날짜는 매칭되는 날짜가 없을 수 있으므로, 가장 근사값을 구하기 위한 방법)
                    last_month = date[j]  # 전월일로 설정

            for j in range(0, date.index(date[i - 1]) - date.index(last_month)):  # 전 거래일부터 전월일까지 돌면서
                if nasdaq_return[i - 1 - j] <= -3:  # 날짜의 인덱스번호로 나스닥 수익률을 추적해서 -3%가 뜨면,
                    drop_count = drop_count + 1  # 하락 카운트 추가

            if drop_count < 3:  # 한달간 -3%가 3번 미만이면
                depression_signal = depression_signal + 1  # 공황 시그널 1추가
                return_date = date[i] + relativedelta(months=1)  # 탈출 날짜 기준 한 달 뒤 복귀 날짜 설정
                market_status.append('평화')
            else: # 한달간 -3%가 3번이면
                depression_signal = depression_signal + 1  # 공황 시그널 1추가 = 공황 시그널 4 (한 달간 -3%가 네 번) = 공황 상태 시작
                return_date = date[i] + relativedelta(months=2)  # 탈출 날짜 기준 두 달 뒤 복귀 날짜 설정
                market_status.append('공황')

    else:  # 나스닥이 -3%가 안 뜨면
        if market_status[i - 1] == '공황':  # 전 날이 공황상태일 때 
            if recovery_signal == 1 or vix_index[i - 1] < 15:  # 회복 시그널이 뜨거나, vix 지수가 15보다 낮으면
                strategy.append('1등주')  # 1등주 보유
                market_status.append('평화')  # 평화상태로 전환
                recovery_signal = 0  # 회복시그널 초기화
            else:  # 회복 시그널이 아니거나, vix 지수가 15보다 높으면
                strategy.append('채권')  # 채권 보유
                market_status.append('공황')
        else:  # 전 날이 평화상태일 때 1등주/채권 선택
            if date[i] < return_date + relativedelta(days=1):  # 복귀 날짜가 안됐을 때
                strategy.append('채권')  # 채권 보유
                market_status.append('평화')
            else:  # 복귀 날짜가 됐을 때
                strategy.append('1등주')  # 1등주 보유
                market_status.append('평화')
                return_date = date[i]

    recovery.append(recovery_signal)
    depression.append(depression_signal)
    return_day.append(return_date)

# 데이터 프레임 만들기
backtest_df = pd.DataFrame()
backtest_df['date'] = date
backtest_df.set_index('date', inplace=True)
backtest_df['nasdaq'] = nasdaq_price
backtest_df['nasdaq_return'] = nasdaq_return
backtest_df['vix'] = vix_index
backtest_df['strategy'] = strategy
backtest_df['market_status'] = market_status
backtest_df['recovery_signal'] = recovery
backtest_df['depression_signal'] = depression
backtest_df['return_date'] = return_day

In [7]:
# 보유종목 구하기
holding_asset = []

# 1일차 보유 종목
if strategy[0] == '채권':  # 채권 보유 전략일 때
    holding_asset.append('treasury')  # 채권 보유
elif strategy[0] == '1등주':  # 1등주 보유 전략일 때
    holding_asset.append(rank1[0])  # 1등주 보유
else:  # 탈출 시그널일 때
    holding_asset.append(rank1[0])  # 1등주 보유

# 2일차 보유 종목
if strategy[1] == '채권':  # 채권 보유 전략일 때
    holding_asset.append('treasury')  # 채권 보유
elif strategy[1] == '1등주':  # 1등주 보유 전략일 때
    holding_asset.append(rank1[1])  # 1등주 보유
else:  # 탈출 시그널일 때
    holding_asset.append(holding_asset[0])  # 전일 보유 종목 그대로 보유

# 나머지 보유 종목
for i in range(2, len(date)):
    if strategy[i] == '채권':  # 채권 보유 전략일 때
        holding_asset.append('treasury')  # 채권 보유

    elif strategy[i] == '1등주':  # 1등주 보유 전략일 때
        if holding_asset[i - 1] == 'treasury' and strategy[i - 1] != '1등주':  # 전일 전략이 1등주가 아니고, 채권 보유하고 있을 때
            holding_asset.append(rank1[i - 1])  # 전일 1등주만 보유

        elif type(holding_asset[i - 1]) == str:  # 보유종목이 string일 때 = 한 종목만 보유하고 있을 때
            if rank1[i - 1] != rank1[i - 2]:  # 1등주가 바뀐 경우
                holding_asset.append((rank1[i - 1], rank2[i - 1]))  # 전일 1등주와 2등주를 모두 보유
            else:
                holding_asset.append(holding_asset[i - 1])  # 전날 보유하고 있던 것 그대로 보유

        else:  # 보유종목이 tuple일 때 = 두 종목을 보유하고 있을 때 = 1등주와 2등주를 모두 보유하고 있을 때
            if rank_distance[i - 1] >= 10:  # 1등주와 2등주가 10% 이상 차이가 나면
                holding_asset.append(rank1[i - 1])  # 전일 1등주만 보유
            else:
                holding_asset.append((rank1[i - 1], rank2[i - 1]))  # 전일 1등주와 2등주 모두 보유

    else:  # 탈출 시그널일 때
        if market_status[i - 1] == '공황':  # 전일이 공황상태이면
            holding_asset.append('treasury')  # 채권 보유
        else:  # 전일이 평화상태이면
            holding_asset.append(holding_asset[i - 1])  # 전날 보유하고 있던 것 그대로 보유

backtest_df['holding_asset'] = holding_asset
print('보유종목 데이터 생성 완료')

보유종목 데이터 생성 완료


In [8]:
# 평가액, 수익률, 누적수익률, 전고점, 낙폭, MDD 계산
balance = [] # 평가액 리스트
balance_tmp = 1000  # 최초 설정액
day_return = [] # 일일 수익률 리스트
cum_return = [] # 누적 수익률 리스트
prev_high = [] # 전고점 리스트
drawdown = [] # 낙폭 리스트
MDD = [] # MDD 리스트

# 1일차
balance_tmp = balance_tmp * (1 + return_df.loc[date[0], holding_asset[0] + '_return'] / 100) # 첫 거래일 보유종목의 수익률로 평가액 계산
balance.append(balance_tmp) # 평가액 리스트에 추가

day_return.append(return_df.loc[date[0], holding_asset[0] + '_return'])  # 보유 종목의 첫 거래일 수익률로 수익률 리스트에 추가
cum_return.append(day_return[0])  # 전략 첫 거래일 수익률로 누적수익률 리스트에 추가

prev_high.append(cum_return[0] if cum_return[0] > 0 else 0)  # 전략 첫 거래일 수익률이 0보다 크면 첫 거래일 수익률로, 아니면 0으로
high = prev_high[0]
drawdown.append(cum_return[0] if cum_return[0] < 0 else 0)  # 전략 첫 거래일 수익률이 0보다 작으면 첫 거래일 수익률로, 아니면 0으로
MDD.append(drawdown[0])  # 첫 거래일 낙폭으로 MDD 리스트에 추가
MDD_tmp = MDD[0]

balance_tmp1 = 0 # 두 종목 동시 보유시 1위 종목용 임시 변수
balance_tmp2 = 0 # 두 종목 동시 보유시 2위 종목용 임시 변수

# 나머지 거래일
for i in range(1, len(date)):
    if isinstance(holding_asset[i], tuple) == False:  # 종목을 하나만 보유하고 있을 때
        balance_tmp = balance_tmp * (1 + return_df.loc[date[i], holding_asset[i] + '_return'] / 100) # 보유종목의 수익률로 전략 평가액 계산
        balance.append(balance_tmp)

    else:  # 1등주와 2등주를 모두 보유하고 있을 때
        if isinstance(holding_asset[i - 1], tuple) == False:  # 처음으로 1등주와 2등주를 동시에 보유하게 됐을 때
            balance_tmp1 = balance[i - 1] / 2
            balance_tmp2 = balance[i - 1] / 2
            balance_tmp1 = balance_tmp1 * (1 + return_df.loc[date[i], holding_asset[i][0] + '_return'] / 100)
            balance_tmp2 = balance_tmp2 * (1 + return_df.loc[date[i], holding_asset[i][1] + '_return'] / 100)
            balance_tmp = balance_tmp1 + balance_tmp2
            holding_asset_tmp1 = holding_asset[i][0] # 1위 종목명 임시 저장
            holding_asset_tmp2 = holding_asset[i][1] # 2위 종목명 임시 저장
            balance.append(balance_tmp)
        else:  # 이전에도 1등주와 2등주를 모두 보유하고 있을 때
            balance_tmp1 = balance_tmp1 * (1 + return_df.loc[date[i], holding_asset_tmp1 + '_return'] / 100)
            balance_tmp2 = balance_tmp2 * (1 + return_df.loc[date[i], holding_asset_tmp2 + '_return'] / 100)
            balance_tmp = balance_tmp1 + balance_tmp2
            balance.append(balance_tmp)

    day_return.append((balance[i] / balance[i - 1] - 1) * 100)  # 전일 평가액 대비 당일 평가액으로 일일 수익률 계산

    cum_return.append((balance[i] / 1000 - 1) * 100)  # 설정액 대비 평가액으로 누적 수익률 계산

    if high < cum_return[i]:  # 누적수익률이 이전고점보다 높으면
        high = cum_return[i]  # 전고점 갱신
    else:
        high = high
    prev_high.append(high)  # 전고점 추가

    drawdown.append(((cum_return[i] + 100) / (prev_high[i] + 100) - 1) * 100)  # 전고점 대비 누적 수익률로 낙폭 계산

    if MDD_tmp > drawdown[i]:  # 낙폭이 이전 MDD보다 작으면
        MDD_tmp = drawdown[i]  # MDD 갱신
    else:
        MDD_tmp = MDD_tmp
    MDD.append(MDD_tmp)

    print(date[i], strategy[i], market_status[i], rank1[i], rank2[i], holding_asset[i], balance[i], day_return[i],
          cum_return[i], prev_high[i], drawdown[i], MDD[i])

backtest_df['balance'] = balance
backtest_df['return'] = day_return
backtest_df['cum_return'] = cum_return
backtest_df['high'] = prev_high
backtest_df['drawdown'] = drawdown
backtest_df['MDD'] = MDD

print('성과 데이터 생성 완료')

2000-01-03 00:00:00 1등주 평화 MSFT GE MSFT 998.3942169811504 -0.16057830188496158 -0.16057830188496158 0 -0.16057830188497269 -0.16057830188497269
2000-01-04 00:00:00 탈출 평화 MSFT GE MSFT 964.6680884388614 -3.378037249080512 -3.5331911561138574 0 -3.5331911561138574 -3.5331911561138574
2000-01-05 00:00:00 채권 평화 MSFT GE treasury 955.6716137238595 -0.9325979394178008 -4.432838627614045 0 -4.432838627614045 -4.432838627614045
2000-01-06 00:00:00 탈출 평화 MSFT GE treasury 962.6694687323034 0.7322447279956457 -3.7330531267696587 0 -3.7330531267696587 -4.432838627614045
2000-01-07 00:00:00 채권 평화 MSFT GE treasury 966.6680337213759 0.4153621901334503 -3.3331966278624114 0 -3.3331966278624114 -4.432838627614045
2000-01-10 00:00:00 채권 평화 MSFT GE treasury 963.6694413735605 -0.31019876971328797 -3.6330558626439524 0 -3.6330558626439524 -4.432838627614045
2000-01-11 00:00:00 탈출 평화 MSFT GE treasury 954.6720596855357 -0.9336585038123069 -4.53279403144643 0 -4.53279403144643 -4.53279403144643
2000-01-12 00:00

In [9]:
# 벤치마크 평가액, 수익률, 누적수익률, 전고점, 낙폭, MDD 계산
benchmark = ['nasdaq', 'treasury', 'rank1', 'rank2'] 

for i in benchmark:
    benchmark_balance = i + '_balance'
    benchmark_return = i + '_return'
    benchmark_cum_return = i + '_cum_return'
    benchmark_prev_high = i + '_prev_high'
    benchmark_drawdown = i + '_drawdown'
    benchmark_MDD = i + '_MDD'

    locals()[benchmark_balance] = []
    locals()[benchmark_cum_return] = []
    locals()[benchmark_prev_high] = []
    locals()[benchmark_drawdown] = []
    locals()[benchmark_MDD] = []

    # 1일차
    balance_tmp = 1000 * (1 + locals()[benchmark_return][0] / 100)
    locals()[benchmark_balance].append(balance_tmp)
    locals()[benchmark_cum_return].append(locals()[benchmark_return][0])
    locals()[benchmark_prev_high].append(locals()[benchmark_cum_return][0] if locals()[benchmark_cum_return][
                                                                                  0] > 0 else 0)  # 첫 거래일이 0보다 크면 첫 거래일 수익률로, 아니면 0으로
    high = locals()[benchmark_prev_high][0]
    locals()[benchmark_drawdown].append(locals()[benchmark_cum_return][0] if locals()[benchmark_cum_return][
                                                                                 0] < 0 else 0)  # 첫 거래일이 0보다 작으면 첫 거래일 수익률로, 아니면 0으로
    locals()[benchmark_MDD].append(locals()[benchmark_drawdown][0])  # 첫 거래일 낙폭으로
    MDD_tmp = locals()[benchmark_MDD][0]

    # 나머지 거래일 
    for i in range(1, len(date)):
        locals()[benchmark_balance].append(
            locals()[benchmark_balance][i - 1] * (1 + locals()[benchmark_return][i] / 100))

        locals()[benchmark_cum_return].append((locals()[benchmark_balance][i] / 1000 - 1) * 100)  # 누적 수익률 계산해서 추가

        if high < locals()[benchmark_cum_return][i]:  # 누적수익률이 이전고점보다 높으면
            high = locals()[benchmark_cum_return][i]  # 전고점 갱신
        else:
            high = high
        locals()[benchmark_prev_high].append(high)  # 전고점 추가

        locals()[benchmark_drawdown].append(((locals()[benchmark_cum_return][i] + 100) / (
                    locals()[benchmark_prev_high][i] + 100) - 1) * 100)  # 낙폭 계산해서 추가

        if MDD_tmp > locals()[benchmark_drawdown][i]:  # 낙폭이 이전 MDD보다 작으면
            MDD_tmp = locals()[benchmark_drawdown][i]  # MDD 갱신
        else:
            MDD_tmp = MDD_tmp
        locals()[benchmark_MDD].append(MDD_tmp)

benchmark_df = pd.DataFrame()
benchmark_df['date'] = date
benchmark_df.set_index('date', inplace=True)

for i in benchmark:
    benchmark_df[i + '_balance'] = locals()[i + '_balance']
for i in benchmark:
    benchmark_df[i + '_cum_return'] = locals()[i + '_cum_return']
for i in benchmark:
    benchmark_df[i + '_prev_high'] = locals()[i + '_prev_high']
for i in benchmark:
    benchmark_df[i + '_drawdown'] = locals()[i + '_drawdown']
for i in benchmark:
    benchmark_df[i + '_MDD'] = locals()[i + '_MDD']

print('벤치마크 데이터 생성 완료')

벤치마크 데이터 생성 완료


In [10]:
time_period = (date[-1] - date[0]).days / 365  # 총 거래 기간 계산
CAGR = ((balance[-1] / 1000) ** (1 / float(time_period)) - 1) * 100  # CAGR 계산 = (최종 평가액/최초 평가액) ^ (1/거래기간) - 1 

for i in benchmark:
    locals()[i + '_CAGR'] = ((locals()[i + '_balance'][-1] / 1000) ** (1 / float(time_period)) - 1) * 100

print('기간 : ' + str(time_period) + '년')
print('### 1등주 전략 ###')
print('누적수익률 : ' + str(cum_return[-1]) + '%')
print('CAGR : ' + str(CAGR) + '%')
print('MDD : ' + str(MDD[-1]) + '%')
for i in benchmark:
    print('### ' + i + ' ###')
    print('누적수익률 : ' + str(locals()[i + '_cum_return'][-1]) + '%')
    print('CAGR : ' + str(locals()[i + '_CAGR']) + '%')
    print('MDD : ' + str(locals()[i + '_MDD'][-1]) + '%')

# csv 파일 생성하기
backtest_df = pd.merge(backtest_df, benchmark_df, left_index=True, right_index=True)
backtest_df = pd.merge(backtest_df, marketcap_ranking, left_index=True, right_index=True)
backtest_df = pd.merge(backtest_df, price_df, left_index=True, right_index=True)
backtest_df = pd.merge(backtest_df, return_df, left_index=True, right_index=True)
backtest_df = pd.merge(backtest_df, marketcap_df, left_index=True, right_index=True)
backtest_df.to_csv('champion_investing.csv', encoding='euc-kr')
print('csv 파일 생성 완료')

기간 : 20.175342465753424년
### 1등주 전략 ###
누적수익률 : 781.0360781162462%
CAGR : 11.388162827825155%
MDD : -44.51941606313731%
### nasdaq ###
누적수익률 : 110.53618411540201%
CAGR : 3.7590141883195427%
MDD : -77.93238628593402%
### treasury ###
누적수익률 : 352.55852638118535%
CAGR : 7.770231670660577%
MDD : -18.42162143869661%
### rank1 ###
누적수익률 : 1792.673312575036%
CAGR : 15.69080266801448%
MDD : -50.71195620674735%
### rank2 ###
누적수익률 : 2975.060501607694%
CAGR : 18.507583297461714%
MDD : -55.914001022447415%
csv 파일 생성 완료
